In [1]:
#Import all dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## CSV Import

First we need to import all necessary csv files to be introduced into the database

In [2]:
csv_file_crunchy = "crunchyroll.csv"
csv_file_mal = "myanimelist.csv"
crunchy_df = pd.read_csv(csv_file_crunchy ) #crunchy roll dataset
mal_df = pd.read_csv(csv_file_mal) #my anime list dataset


# Cleaning the datasets, and first transformations

In the cleaning process I need to drop duplicates from each dataset to be used, as I am interested only in one iteration of the anime pero dataset. 

Also i had to put everything in lowercase, and remove any special character that was not a-z or 0-9. 

For the last step I standardized the scores to 0-100, as for crunchy roll it is on a scale of 1-5 and for my anime list it is in a scale of 1-10. 

In [3]:
#Drop duplicates in column title, lower de cases, and remove non alphabeticals, and normalize rate, and score
crunchy_df.drop_duplicates(subset ="anime", 
                     keep = "first", inplace = True) 
mal_df.drop_duplicates(subset ="title", 
                     keep ="first" , inplace = True) 

#make titles lower-case
crunchy_df["anime"]=crunchy_df['anime'].str.lower()
mal_df["title"]=mal_df['title'].str.lower()

#remove special characters
crunchy_df["anime"]=  crunchy_df['anime'].astype(str).str.replace('[^a-z,0-9," "]', '')
mal_df["title"]=mal_df['title'].astype(str).str.replace('[^a-z,0-9," "]', '')

#Standardize the ratings
crunchy_df['rate']= crunchy_df['rate']/5 *100
mal_df['score']=mal_df['score']/10 *100

Next two cell I just check the values of the columns to decide with which ones I am going to stay for the tables to incorporate in my database.

In [4]:
#Check column values for crunchy roll
crunchy_df.columns.values

array(['anime', 'anime_url', 'anime_img', 'episodes', 'votes', 'weight',
       'rate', 'rate_1', 'rate_2', 'rate_3', 'rate_4', 'rate_5',
       'genre_action', 'genre_adventure', 'genre_comedy', 'genre_drama',
       'genre_family', 'genre_fantasy', 'genre_food', 'genre_harem',
       'genre_historical', 'genre_horror', 'genre_idols', 'genre_isekai',
       'genre_jdrama', 'genre_magical girls', 'genre_martial arts',
       'genre_mecha', 'genre_music', 'genre_mystery',
       'genre_post-apocalyptic', 'genre_romance', 'genre_sci-fi',
       'genre_seinen', 'genre_sgdrama', 'genre_shojo', 'genre_shonen',
       'genre_slice of life', 'genre_sports', 'genre_supernatural',
       'genre_thriller'], dtype=object)

In [5]:
#Check column values for my anime list
mal_df.columns.values

array(['uid', 'title', 'synopsis', 'genre', 'aired', 'episodes',
       'members', 'popularity', 'ranked', 'score', 'img_url', 'link'],
      dtype=object)

I decided to use genre as an important characteristic of each anime, however in both my anime list and crunchy rolls datasets the formats for their genres are different. I needed to standardize them.

For the Crunchy Roll as each genre is represented as a column value I decided to unpivot them with a melt, so I could have individual records.

For the My Anime List as each genre is represented as a list for each entry I decided to desaggregate them into different entries.

In [6]:
#Unpivot/desaggregate the genres on the crunchy roll dataset
crunchy_df_2 = pd.melt(crunchy_df, id_vars=['anime', 'anime_url','episodes','rate'], value_vars=['genre_action', 'genre_adventure', 'genre_comedy', 'genre_drama',
       'genre_family', 'genre_fantasy', 'genre_food', 'genre_harem',
       'genre_historical', 'genre_horror', 'genre_idols', 'genre_isekai',
       'genre_jdrama', 'genre_magical girls', 'genre_martial arts',
       'genre_mecha', 'genre_music', 'genre_mystery',
       'genre_post-apocalyptic', 'genre_romance', 'genre_sci-fi',
       'genre_seinen', 'genre_sgdrama', 'genre_shojo', 'genre_shonen',
       'genre_slice of life', 'genre_sports', 'genre_supernatural',
       'genre_thriller']) # new crunchy roll dataset



In [7]:
#Desaggregate each list of genres on the My Anime List dataset: 
#for each row make each list element on the column genre in my anime list data set a datapoint
#Important observation: each "list" of genres in reality was a string with characters that made it look like a list,
#I had to work around a way to separate the elements on this list.
title=[]
episodes = []
score =[]
link= []
genre=[]
for index, data in mal_df.iterrows():
  
   
     
    time = data['genre'] 
    time=time.replace('[', '') 
    time=time.replace(']','')
    time= time.split(',')
    
    for row in time:
        genre.append(row)
        title.append(data['title'])
        episodes.append(data['episodes'])
        score.append(data['score'])
        link.append(data['link'])
        


In the last step I created 5 list elements: title, genre, episodes, score, and link. Now I will bundle them up on a dictionary, and convert it to my new dataset for my anime list

In [8]:
dict = {'title': title,'link':link,'episodes': episodes,'score':score, 'genre':genre} 
mal_df_2 = pd.DataFrame(dict) #new my anime list dataset

In [9]:
#Check my new crunchy roll data set
crunchy_df_2.head(5)


,anime,anime_url,episodes,rate,variable,value
0,naruto shippuuden,https://www.crunchyroll.com/naruto-shippuden,500,96.2,genre_action,0.0
1,shugo chara,https://www.crunchyroll.com/shugo-chara,51,97.2,genre_action,0.0
2,bleach,https://www.crunchyroll.com/bleach,366,96.6,genre_action,0.0
3,naruto,https://www.crunchyroll.com/naruto,220,96.0,genre_action,0.0
4,skip beat,https://www.crunchyroll.com/skip-beat,0,98.2,genre_action,0.0


In [10]:
#Check my new my anime list dataset
mal_df_2.head()

,title,link,episodes,score,genre
0,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,'Comedy'
1,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,'Sports'
2,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,'Drama'
3,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,'School'
4,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,'Shounen'


#  Last transformations to data

Now it is time for the last transformations for both the my anime list, and crunchy roll datasets. 

I want them to have the same titles, drop missing unwanted characters, and unwanted columns. 

Also I added a new column of source.

In [11]:
#Dropping one column from my crunchy rolls dataset which is not needed, replacing the string "genre_" on each value in genre
#and renaming columns.
crunchy_df_2=crunchy_df_2.drop(['value'],axis=1)
crunchy_df_2['variable'] = crunchy_df_2['variable'].str.replace('genre_', '').astype(str)
crunchy_df_2 = crunchy_df_2.rename(columns={"anime": "title","anime_url":"url", "rate":"score","variable":"genre"})


In [12]:
crunchy_df_2.head(5)

,title,url,episodes,score,genre
0,naruto shippuuden,https://www.crunchyroll.com/naruto-shippuden,500,96.2,action
1,shugo chara,https://www.crunchyroll.com/shugo-chara,51,97.2,action
2,bleach,https://www.crunchyroll.com/bleach,366,96.6,action
3,naruto,https://www.crunchyroll.com/naruto,220,96.0,action
4,skip beat,https://www.crunchyroll.com/skip-beat,0,98.2,action


In [13]:
#For my my anime liste dataset: removing unwanted character from genre, and lowercasing them
#Found out some episodes had null values, so I removed them.
#Changed the episode type to int as I wanted to remove the ".0" that was previously displayed
#Renamed the link column.
mal_df_2['genre'] = mal_df_2['genre'].str.replace("'", '').astype(str)
mal_df_2["genre"]=mal_df_2['genre'].str.lower()
mal_df_2 = mal_df_2[mal_df_2['episodes'].notnull()]
mal_df_2['episodes'] = mal_df_2['episodes'].astype(int)
mal_df_2 = mal_df_2.rename(columns={"link":"url"})

In [14]:
mal_df_2.head(5)

,title,url,episodes,score,genre
0,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25,88.2,comedy
1,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25,88.2,sports
2,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25,88.2,drama
3,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25,88.2,school
4,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25,88.2,shounen


As a las step I added the source column to both datasets, and checked that their column data types matched.

In [15]:
#Adding source column
crunchy_df_2['source'] = 'crunchyroll'
mal_df_2['source']='myanimelist'

In [16]:
crunchy_df_2.dtypes

title        object
url          object
episodes      int64
score       float64
genre        object
source       object
dtype: object

In [17]:
mal_df_2.dtypes

title        object
url          object
episodes      int64
score       float64
genre        object
source       object
dtype: object

# Append data frames, merge data frames and reset index

Now that both my datasets are on the same structure I appended them and ingnored their index, and visualized my ready to use dataset.

In [18]:
merge_df =crunchy_df_2.append(mal_df_2, ignore_index=True)
merge_df.shape

(81698, 6)

In [19]:
merge_df.columns

Index(['title', 'url', 'episodes', 'score', 'genre', 'source'], dtype='object')

In [20]:
#ready to use dataframe, from here we should extract the relational tables for sql database
merge_df

,title,url,episodes,score,genre,source
0,naruto shippuuden,https://www.crunchyroll.com/naruto-shippuden,500,96.2,action,crunchyroll
1,shugo chara,https://www.crunchyroll.com/shugo-chara,51,97.2,action,crunchyroll
2,bleach,https://www.crunchyroll.com/bleach,366,96.6,action,crunchyroll
3,naruto,https://www.crunchyroll.com/naruto,220,96.0,action,crunchyroll
4,skip beat,https://www.crunchyroll.com/skip-beat,0,98.2,action,crunchyroll
...,...,...,...,...,...,...
81693,chuunibyou demo koi ga shitai kirameki no slap...,https://myanimelist.net/anime/16934/Chuunibyou...,1,75.6,comedy,myanimelist
81694,chuunibyou demo koi ga shitai kirameki no slap...,https://myanimelist.net/anime/16934/Chuunibyou...,1,75.6,drama,myanimelist
81695,chuunibyou demo koi ga shitai kirameki no slap...,https://myanimelist.net/anime/16934/Chuunibyou...,1,75.6,romance,myanimelist
81696,chuunibyou demo koi ga shitai kirameki no slap...,https://myanimelist.net/anime/16934/Chuunibyou...,1,75.6,school,myanimelist


## Create tables to input into database

Now I create the tables I am going to input into my relational database. The goal of the database is to extract the rating of an anime, and also be able to pin down the genre of it. However both the score to the anime, and its genre assignation depend on the origin/source of the information. For example, an anime may be in both crunchy roll and my anime list, however their score may be different, and also their genres.  

First I made 3 tables which hold the primary keys:

    Anime catalogue: In here it is the compendium of all the animes as unique values in all my data.

    Origin: In here I specify if the origin is either crunchy roll or my anime list.

    Genre: In here I specify the genres in which an anime can fall.

I made the table that holds the ratings:

    Ratings: All the ratings per anime, and the link for the rating.

A table holding the relation of the three primary keys (Anime, origin, and genre)



In [21]:
#Anime Catalogue
Anime_Catalogue_df = merge_df[['title','episodes']].copy()
Anime_Catalogue_df = Anime_Catalogue_df.drop_duplicates('title')
Anime_Catalogue_df = Anime_Catalogue_df.reset_index()
Anime_Catalogue_df = Anime_Catalogue_df.drop(['index'],axis= 1)
Anime_Catalogue_df = Anime_Catalogue_df.reset_index()
Anime_Catalogue_df= Anime_Catalogue_df.rename(columns={'index':'id_anime'})
Anime_Catalogue_df['id_anime']= Anime_Catalogue_df['id_anime']+1
Anime_Catalogue_df.head(5)

,id_anime,title,episodes
0,1,naruto shippuuden,500
1,2,shugo chara,51
2,3,bleach,366
3,4,naruto,220
4,5,skip beat,0


In [22]:
#Origin
Origin_df = merge_df[['source']].copy()
Origin_df  = Origin_df.drop_duplicates('source')
Origin_df  = Origin_df.reset_index()
Origin_df  = Origin_df.drop(['index'],axis= 1)
Origin_df  = Origin_df.reset_index()
Origin_df =  Origin_df.rename(columns={'index':'id_source'})
Origin_df['id_source']= Origin_df['id_source']+1
Origin_df.head(5)

,id_source,source
0,1,crunchyroll
1,2,myanimelist


In [23]:
#genre
genre_df = merge_df[['genre']].copy()
genre_df  = genre_df.drop_duplicates('genre')
genre_df  = genre_df.reset_index()
genre_df  = genre_df.drop(['index'],axis= 1)
genre_df  = genre_df.reset_index()
genre_df =  genre_df.rename(columns={'index':'id_genre'})
genre_df['id_genre']= genre_df['id_genre']+1
genre_df.head(5)

,id_genre,genre
0,1,action
1,2,adventure
2,3,comedy
3,4,drama
4,5,family


I created a general dataset to obtain my rating, and ids dataset from. I aggregated the source, title, and genre into this table to have its ids.

In [45]:
#For ratings, id tables. 
General_df = merge_df.merge(Origin_df, left_on='source', right_on='source', how='left')
General_df = General_df.merge(Anime_Catalogue_df, left_on='title', right_on='title', how='left')
General_df =  General_df.merge(genre_df, left_on='genre', right_on='genre', how='left')
General_df.columns.values

array(['title', 'url', 'episodes_x', 'score', 'genre', 'source',
       'id_source', 'id_anime', 'episodes_y', 'id_genre'], dtype=object)

In [46]:
#ratings
Ratings_df = General_df[['id_anime','id_source','score','url']].copy()
Ratings_df['AS']=Ratings_df.id_source+Ratings_df.id_anime
Ratings_df=Ratings_df.drop_duplicates('AS')
Ratings_df=Ratings_df.drop(['AS'], axis=1)
Ratings_df = Ratings_df.reset_index()
Ratings_df = Ratings_df.drop(['index'],axis= 1)
Ratings_df.head(10)

,id_anime,id_source,score,url
0,1,1,96.2,https://www.crunchyroll.com/naruto-shippuden
1,2,1,97.2,https://www.crunchyroll.com/shugo-chara
2,3,1,96.6,https://www.crunchyroll.com/bleach
3,4,1,96.0,https://www.crunchyroll.com/naruto
4,5,1,98.2,https://www.crunchyroll.com/skip-beat
5,6,1,97.6,https://www.crunchyroll.com/reborn
6,7,1,97.4,https://www.crunchyroll.com/gintama
7,8,1,97.2,https://www.crunchyroll.com/la-corda-doro-prim...
8,9,1,97.0,https://www.crunchyroll.com/eyeshield-21
9,10,1,97.0,https://www.crunchyroll.com/hayate-the-combat-...


In [47]:
#Anime, Genre, Source as foreign keys table
AGS_df = General_df[['id_anime','id_genre','id_source']].copy()
AGS_df['AGS']=AGS_df.id_anime+AGS_df.id_genre+AGS_df.id_source
AGS_df=AGS_df.drop_duplicates('AGS')
AGS_df=AGS_df.drop(['AGS'], axis=1)
AGS_df

,id_anime,id_genre,id_source
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,1
...,...,...,...
81676,16521,35,2
81678,16522,74,2
81683,16523,48,2
81684,16523,67,2


# Connect to database,and export created tables

As a last step I connect my tables to the database. The following schema is what I used in sql to create my tables:

    CREATE TABLE anime_catalogue (
    id_anime INT PRIMARY KEY,
    title VARCHAR(200),
    episodes INT
    );

    CREATE TABLE origin (
    id_source INT PRIMARY KEY,
    source VARCHAR(200)
    );

    CREATE TABLE genre (
    id_genre INT PRIMARY KEY,
    genre VARCHAR(200)
    );

    CREATE TABLE ratings(
        id_anime INT,
        id_source INT,
        score FLOAT,
        url VARCHAR(200),
        FOREIGN KEY (id_anime) REFERENCES anime_catalogue (id_anime),
        FOREIGN KEY (id_source) REFERENCES origin(id_source)
    )

    CREATE TABLE anime_genre_source(
        id_anime INT,
        id_genre INT,
        id_source INT,
        FOREIGN KEY (id_anime) REFERENCES anime_catalogue (id_anime),
        FOREIGN KEY (id_genre) REFERENCES genre(id_genre),
        FOREIGN KEY (id_source) REFERENCES origin(id_source)
    )


In [48]:
rds_connection_string = "postgres:ihatefred@localhost:5432/Anime_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [49]:
engine.table_names()

['anime_catalogue', 'origin', 'anime_genre_source', 'genre', 'ratings']

In [50]:
Anime_Catalogue_df.to_sql(name='anime_catalogue', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "anime_catalogue_pkey"
DETAIL:  Key (id_anime)=(1) already exists.

[SQL: INSERT INTO anime_catalogue (id_anime, title, episodes) VALUES (%(id_anime)s, %(title)s, %(episodes)s)]
[parameters: ({'id_anime': 1, 'title': 'naruto shippuuden', 'episodes': 500}, {'id_anime': 2, 'title': 'shugo chara', 'episodes': 51}, {'id_anime': 3, 'title': 'bleach', 'episodes': 366}, {'id_anime': 4, 'title': 'naruto', 'episodes': 220}, {'id_anime': 5, 'title': 'skip beat', 'episodes': 0}, {'id_anime': 6, 'title': 'reborn', 'episodes': 0}, {'id_anime': 7, 'title': 'gintama', 'episodes': 379}, {'id_anime': 8, 'title': 'la corda doro primo passo and secondo passo', 'episodes': 0}  ... displaying 10 of 16526 total bound parameter sets ...  {'id_anime': 16525, 'title': 'kingsglaive final fantasy xv', 'episodes': 1}, {'id_anime': 16526, 'title': 'chuunibyou demo koi ga shitai kirameki no slapstick noel', 'episodes': 

In [31]:
Origin_df.to_sql(name='origin', con=engine, if_exists='append', index=False)

In [32]:
genre_df.to_sql(name='genre', con=engine, if_exists='append', index=False)

In [43]:
Ratings_df.to_sql(name='ratings', con=engine, if_exists='append', index=False)

In [35]:
AGS_df.to_sql(name='anime_genre_source', con=engine, if_exists='append', index=False)

# Test to see if tables where copied correctly to db

As a last step I just check that the connection was made succesfully for all my tables.

In [51]:
pd.read_sql_query('select * from anime_catalogue', con=engine).head()

,id_anime,title,episodes
0,1,naruto shippuuden,500
1,2,shugo chara,51
2,3,bleach,366
3,4,naruto,220
4,5,skip beat,0


In [52]:
pd.read_sql_query('select * from ratings', con=engine).head()

,id_anime,id_source,score,url
0,1,1,96.2,https://www.crunchyroll.com/naruto-shippuden
1,2,1,97.2,https://www.crunchyroll.com/shugo-chara
2,3,1,96.6,https://www.crunchyroll.com/bleach
3,4,1,96.0,https://www.crunchyroll.com/naruto
4,5,1,98.2,https://www.crunchyroll.com/skip-beat


In [53]:
pd.read_sql_query('select * from genre', con=engine).head()

,id_genre,genre
0,1,action
1,2,adventure
2,3,comedy
3,4,drama
4,5,family


In [54]:
pd.read_sql_query('select * from origin', con=engine).head()

,id_source,source
0,1,crunchyroll
1,2,myanimelist


In [55]:
pd.read_sql_query('select * from anime_genre_source', con=engine).head()

,id_anime,id_genre,id_source
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,1
